In [1]:
#essential libraries

#from google_drive import gdrive

from matplotlib.pyplot import show
import numpy as np
import pandas as pd
from scipy.cluster.vq import *
import operator
import matplotlib
reload(matplotlib)
matplotlib.use('Agg')
from matplotlib import pyplot as plt
import pickle
import shelve
import re
from collections import Counter, defaultdict, OrderedDict, deque
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
import scipy
from sklearn.feature_extraction import DictVectorizer
from matplotlib.backends.backend_pdf import PdfPages
import csv
import sys
import math
from copy import deepcopy, copy
import random
import rosetta
from datetime import datetime, timedelta
from operator import itemgetter
from itertools import chain
import os

import pprint
pp = pprint.PrettyPrinter()

import rdflib
from rdflib.namespace import OWL, RDF, RDFS
from rdflib import URIRef

from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import OneClassSVM, SVC
from sklearn.mixture import GMM
from sklearn.mixture import DPGMM
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage, dendrogram
import scipy.cluster.hierarchy as hier
from scipy.cluster.hierarchy import fcluster
from scipy import spatial
from sklearn.multiclass import OneVsRestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics
from scipy.cluster.hierarchy import linkage, dendrogram
import scipy.cluster.hierarchy as hier

from scipy.spatial.distance import cosine as cosine_similarity

from divergence import gau_js as js_divergence
import building_tokenizer as toker
import brick_parser
reload(brick_parser)
from brick_parser import tagList, tagsetList, equipTagsetList, pointTagsetList, locationTagsetList,\
equalDict, pointTagList, equipTagList, locationTagList, equipPointDict
subTagListDict = dict([('point', pointTagList),
                          ('equip', equipTagList),
                          ('location', locationTagList)
                         ])
subTagsetListDict = dict([('point', pointTagsetList),
                          ('equip', equipTagsetList),
                          ('location', locationTagsetList)
                         ])

debugFlag = False

Init Graph
Load Brick.ttl
Load BrickFrame.ttl
Init Graph
Load Brick.ttl
Load BrickFrame.ttl


/home/jbkoh/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
def pick_dict_with_best_n_from_dict(srcDict, n):
    idxCnt = 0
    sortedValueList = sorted(srcDict.values(), reverse=True)[0:n]
    chosenItemList = list()
    
    for score in sortedValueList:
        for key, val in srcDict.items():
            if idxCnt>=n:
                break
            if val==score:
                chosenItemList.append((key,val))
                idxCnt += 1
        if idxCnt>=n:
                break
    return OrderedDict(chosenItemList)

def save_fig(fig, name, dpi=400):
	pp = PdfPages(name)
	pp.savefig(fig, bbox_inches='tight', pad_inches=0, dpi=dpi)
	pp.close()

buildingName = 'ebu3b'

naeDict = dict()
naeDict['bonner'] = ["607", "608", "609", "557", "610"]
naeDict['ap_m'] = ['514', '513','604']
naeDict['bsb'] = ['519', '568', '567', '566', '564', '565']
naeDict['ebu3b'] = ["505", "506"]
naeList = naeDict[buildingName]

labeledFile = 'metadata/' + buildingName + '_sensor_types_location.csv'
with open(labeledFile, 'rb') as fp:
    #truthDF = pd.read_excel(fp)
    truthDF = pd.DataFrame.from_csv(fp)
    #truthDF = truthDF.set_index(keys='Unique Identifier')

wordFeatFile = 'data/wordfeat_'+buildingName+'.pkl'

tokenTypeList = ['NoNumber', 'Alphanumeric', 'AlphaAndNum', 'NumAsSingleWord']

bacnetTypeMapDF = pd.DataFrame.from_csv('metadata/bacnettype_mapping.csv')
unitMap = pd.read_csv('metadata/unit_mapping.csv').set_index('unit')
for val in Counter(unitMap.keys()).values():
    if val>1:
        "Unit map file ERROR"
        assert(False)
        
tokenType = 'NoNumber'
trueDF = pd.DataFrame.from_csv('metadata/'+buildingName+'_sensor_types_location.csv')
sensorDF, nameList, jcinameList, descList, unitList, wordList = \
toker.structure_metadata(buildingName=buildingName, tokenType=tokenType, \
                         validSrcidList=trueDF.index.tolist(), withDotFlag=False)


In [5]:
sensorDF.keys()

Index([u'description', u'jci_name', u'name', u'type', u'type_string', u'unit'], dtype='object')

In [6]:
encoderDict = {'A':0, '1':1, '/':2, ' ':3, '.':4}
underbarAdder = lambda x,y:str(x) + '_' + str(y)
listMerger = lambda x:reduce(underbarAdder,x, '')

structListDict = dict()
structStrListDict = dict()

for colName, col in sensorDF[['description', u'jci_name', u'name']].iteritems():
    structList = list()
    for srcid, sentence in col.iteritems():
        newStr = []
        for c in sentence:
            if re.match('[a-zA-Z]', c):
                newStr.append(0)
            elif re.match('\d+', c):
                newStr.append(1)
            else:
                if c in encoderDict.keys():
                    newStr.append(encoderDict[c])
                else:
                    encoderDict[c] = max(encoderDict.values())+1
                    newStr.append(encoderDict[c])
        removeIdx = [i+1 for i in range(0,len(newStr)-1) if newStr[i]==newStr[i+1]]
        removeIdx.reverse()
        for idx in removeIdx:
            #newStr = newStr[:idx] + newStr[idx+1:]
            del(newStr[idx])
        structList.append(newStr)
    structListDict[colName] = structList
    structStrListDict[colName] = map(listMerger, structList)
for colName, structStrList in structStrListDict.items():
    sensorDF[colName+'_struct'] = structStrList

In [22]:
subDF = sensorDF[sensorDF['name_struct']=='_0_3_1_3_0_1_3_1_3_0_1_3_0_3_0']['name']
for row in subDF.tolist():
    print row, '\t', 
    

source_id
506_0_3000023         NAE 06 N2 1 VMA102 ZN T
506_0_3000028     NAE 06 N2 1 VMA102 DMPR POS
506_0_3000029     NAE 06 N2 1 VMA102 SUP FLOW
506_0_3000044         NAE 06 N2 1 VMA127 ZN T
506_0_3000049     NAE 06 N2 1 VMA127 DMPR POS
506_0_3000050     NAE 06 N2 1 VMA127 SUP FLOW
506_0_3000064         NAE 06 N2 1 VMA128 ZN T
506_0_3000069     NAE 06 N2 1 VMA128 DMPR POS
506_0_3000070     NAE 06 N2 1 VMA128 SUP FLOW
506_0_3000084         NAE 06 N2 1 VMA129 ZN T
506_0_3000089     NAE 06 N2 1 VMA129 DMPR POS
506_0_3000090     NAE 06 N2 1 VMA129 SUP FLOW
506_0_3000104         NAE 06 N2 1 VMA130 ZN T
506_0_3000109     NAE 06 N2 1 VMA130 DMPR POS
506_0_3000110     NAE 06 N2 1 VMA130 SUP FLOW
506_0_3000124         NAE 06 N2 1 VMA131 ZN T
506_0_3000129     NAE 06 N2 1 VMA131 DMPR POS
506_0_3000130     NAE 06 N2 1 VMA131 SUP FLOW
506_0_3000144         NAE 06 N2 1 VMA132 ZN T
506_0_3000149     NAE 06 N2 1 VMA132 DMPR POS
506_0_3000150     NAE 06 N2 1 VMA132 SUP FLOW
506_0_3000164         NA

In [ ]:
oneCharSet = set()
for colName, col in sensorDF[['description', u'jci_name', u'name']].iteritems():
    for col.i

In [1]:
oneCharSet = set()
for colName, col in sensorDF[['description', u'jci_name', u'name']].iteritems():
    for sentence in col.tolist():
        wordList = re.findall('[a-zA-Z]+', sentence)
        for word in wordList:
            if len(word)==1:
                oneCharSet.add(word)
                if word=='N':
                    print sentence

NameError: name 'sensorDF' is not defined